# Importing Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from math import pi,acos,log10
import os
import seaborn as sns
from scipy.constants import epsilon_0 as epsilon
%matplotlib inline

In [ ]:
#Run this to reset the results
results= pd.DataFrame(columns = ['LC Name', 'Temperature', 'Applied Voltage','Rise Time','Decay Time','Total Response Time','Visco-Elastic Ratio'])

In [ ]:
#Loading the data
path="/home/avinash/Desktop/Projects/Liquid_Crystal_Analysis/data/Batch4/Electro-optic/Doped" #Enter Loaction WHere CSV data is kept


name_of_LC="Doped"# Change this to show type of LC
thickness= 6.5E-6 #in Metres
area=70E-6 #in Metres^2
empty_c=102E-12 #in Farad
wavelength=633E-9 #in metre
#Dielectric Anisotropy depends on temperature

voltages=['5','6','7','8','9','10']
#epsilon_a=[1,1,1,1]

if not os.path.exists(path + "/Analysis") :
    os.mkdir (path + "/Analysis")
output_path=path+"/Analysis"

# Finding the Threshold Voltage

In [ ]:
#Reading the datafile containing the Threshold Voltage data
vthdata=pd.read_csv(path+"/Doped_Voltage transmission_RT.csv",skiprows=0)
#vthdata.columns(['Voltage','Intensity'])
#Normalizing Voltage
#vthdata=vthdata.rename(columns={"v":"Voltage","intensity":"Intensity"})
vthdata['Intensity']=(vthdata['Intensity']-vthdata['Intensity'].min())/(vthdata['Intensity'].max()-vthdata['Intensity'].min())

vthdata

In [ ]:
#Plotting the normalized intensity vs voltage data
plt.figure(2,figsize=(12, 8), dpi=100, linewidth=0.1)
plt.ylabel("Normalized Intensity")
plt.xlabel("Applied Voltage (in Volts)")
plt.plot(vthdata['Voltage'],vthdata['Intensity'],'-o')
#indx_max = np.argmax(vthdata['Intensity'])
#Set on your own
indx_max = 45
plt.plot(vthdata['Voltage'][indx_max], vthdata['Intensity'][indx_max], 'ro')
vth=vthdata['Voltage'][indx_max]
#plt.plot(vth*np.ones(2),[1,vthdata['Intensity'][indx_max]])
plt.annotate("Threshold Votage:"+ str(vth)+'V',(vthdata['Voltage'][indx_max], vthdata['Intensity'][indx_max]),xytext=(vthdata['Voltage'][indx_max]+0.25, vthdata['Intensity'][indx_max]))
print("Threshold Votage:"+ str(vth)+'V')
plt.title(name_of_LC+" Threshold_Voltage")
plt.savefig(output_path+"/"+name_of_LC+"_"+"Threshold_Voltage.jpg", dpi=300,bbox_inches='tight')
plt.show()

In [ ]:
#FInding the birefringence from the Intensity vs Voltage Data
#Threshold Voltage depends on temperature of material
plt.figure(2,figsize=(12, 8), dpi=100, linewidth=0.1)
plt.ylabel("Degradation in terms of Birefringence")
plt.xlabel("Applied Voltage (in Volts)")
biref=np.arccos(1-vthdata['Intensity']*2)/2/pi/thickness*wavelength
plt.plot(vthdata['Voltage'],biref,'-o')

#indx_max = np.argmax(biref)
#plt.plot(vthdata['Voltage'][indx_max], biref[indx_max], 'ro')
#b=biref[indx_max]
#plt.plot(vth*np.ones(2),[1,vthdata['Intensity'][indx_max]])
#plt.annotate("Max Birefringence:"+ str(b),(vthdata['Voltage'][indx_max], biref[indx_max]),xytext=(vthdata['Voltage'][indx_max]+0.25, biref[indx_max]))
#print("Max Birefrigence:"+ str(b)+'V')

plt.title(name_of_LC+" Birefringence")
plt.savefig(output_path+"/"+name_of_LC+"_"+"Birefringence.jpg", dpi=300,bbox_inches='tight')
plt.show()

# Analysis of Each Temperature and Voltage Case

In [ ]:
if not os.path.exists(output_path+"/TransmittancevsTime") :
    temp=0
    os.mkdir (output_path+"/TransmittancevsTime")
    for voltage in voltages:
        try :
            #temp=str(temp)
            #voltage=str(voltage)
            if voltage=='10':
                input_path=path+"/"+voltage+"_2.csv"
            else:
                input_path=path+"/"+voltage+"__2.csv"
            df=pd.read_csv(input_path,skiprows=1)
            #Normalizing the Volt to represent transmittance
            df.Volt=(df.Volt-df.Volt.min())/(df.Volt.max()-df.Volt.min())
        except :
            print ("Incorrect CSV directory or filename")
            continue
        else:
            print("File Exists. Start Analysis")
        plt.figure(figsize=(12, 8), dpi=100, linewidth=0.1)
        plt.xlabel("Time (in seconds)")
        plt.ylabel("Transmittance")

        #Have set the window size as a constant = 33. If the number of intersections on
        #the 0.9 line or 0.1 line are more than 6, then icncrease window size
        window_size =33
        smooth=savgol_filter(df.Volt,window_size,2) #Creating a smoothened Curve us SavGol filter with window size 25 

        plt.plot(df.second,smooth) #Printing the smooth curve
        plt.plot(df.second,df.Volt)#Printing the actual curve

        plt.plot(df.second,0.9*np.ones(df.second.shape)) #Printing horizontal line at 0.9
        plt.plot(df.second,0.1*np.ones(df.second.shape)) #Printing horizontal line at 0.1

        #Finding position where the curve closses the 0.9 transmittance
        idx = np.argwhere(np.diff(np.sign(smooth - 0.9))).flatten()
        if(len(idx)!=6):
            print("Error in top. Number of points greater than 6. Increase window size and retry")
            exit(1)
        idx=idx[1:5] #Eliminating First and last points
        plt.plot(df.second[idx], 0.9*np.ones(idx.size), 'ro') #Plotting Points of intersection
        #print(len(idx))
        idx_top=idx
        for i in range(len(idx)):
            plt.annotate("T90="+str(round(df.second[idx[i]],4)),(df.second[idx[i]], 0.9),xytext =(df.second[idx[i]]+0.0005-max(0,0.006*pow(-1,i+1)), 0.91))


        #Finding position where the curve closses the 0.1 transmittance
        idx = np.argwhere(np.diff(np.sign(smooth - 0.1))).flatten()
        if(len(idx)!=6):
            print("Error in bot. Number of points greater than 6. Increase window size and retry")
            exit(1)
        idx=idx[1:5] #Eliminating First and last points
        plt.plot(df.second[idx], 0.1*np.ones(idx.size), 'bo') #Plotting Points of intersection
        #print(len(idx))
        idx_bot=idx
        for i in range(len(idx)):
            plt.annotate("T10="+str(round(df.second[idx[i]],4)),(df.second[idx[i]], 0.1),xytext =(df.second[idx[i]]+0.0005-max(0,0.006*pow(-1,i)), 0.11-max(0,0.05*pow(-1,i))))

        #Calculating RiseTime and DecayTime (average of 2 sets)
        rise_time=((df.second[idx_bot[0]]-df.second[idx_top[0]])+(df.second[idx_bot[2]]-df.second[idx_top[2]]))/2
        decay_time=((df.second[idx_top[1]]-df.second[idx_bot[1]])+(df.second[idx_top[3]]-df.second[idx_bot[3]]))/2
        response_time=rise_time+decay_time
        #Printing the times
        #print("Rise Time = ",rise_time,"seconds")
        #print("Decay Time = ",decay_time,"seconds")
        #print("Response Time = ",response_time,"seconds")
        plt.annotate("Rise Time = "+str(round(rise_time,4)),xy=(0.019,1))
        plt.annotate("Decay Time="+str(round(decay_time,4)),xy=(0.019,0.97))
        plt.annotate("Resp. Time="+str(round(response_time,4)),xy=(0.019,0.94))
        plt.annotate(name_of_LC,xy=(-0.027,1))
        plt.annotate("Appl. Volt. = "+voltage+"V",xy=(-0.027,0.97))
        #plt.annotate("Temp = "+temp+"C",xy=(-0.027,0.94))
        plt.title(name_of_LC+"_"+voltage+"V")
        plt.savefig(output_path+"/TransmittancevsTime/"+voltage+".jpg", dpi=300,bbox_inches='tight')
        #plt.show()
        results=results.append({'LC Name':name_of_LC, 'Temperature':temp, 'Applied Voltage':voltage,'Rise Time':rise_time,'Decay Time':decay_time,'Total Response Time': response_time,'Visco-Elastic Ratio':decay_time*(pi/thickness)**2},ignore_index=True)
        

In [ ]:
#If you are happy with the results, you can write it to a csv file.
results.to_csv(output_path+"/"+"RespTimevsAppVolt.csv", index=False)

In [ ]:
results=pd.read_csv(output_path+"/RespTimevsAppVolt.csv")
results.tail()

In [ ]:
#Plotting the response time vs Applied Voltage
if not os.path.exists(output_path+"/ResponseTimevsTemp") :
    os.mkdir (output_path+"/ResponseTimevsTemp")
temps=['0']
for idx,temp in enumerate(temps):
    plt.figure(1,figsize=(12, 8), dpi=100, linewidth=0.1)
    plt.ylabel("Time (in seconds)")
    plt.xlabel("Applied Voltage (in Volts)")
    plt.plot(results['Applied Voltage'][idx*len(voltages):(idx+1)*len(voltages)],results['Total Response Time'][idx*len(voltages):(idx+1)*len(voltages)],'-o')
    plt.plot(results['Applied Voltage'][idx*len(voltages):(idx+1)*len(voltages)],results['Rise Time'][idx*len(voltages):(idx+1)*len(voltages)],'-o')
    plt.plot(results['Applied Voltage'][idx*len(voltages):(idx+1)*len(voltages)],results['Decay Time'][idx*len(voltages):(idx+1)*len(voltages)],'-o')
    plt.legend(['Total Response Time','Rise Time','Decay Time'])
    plt.title(name_of_LC+" - Response Time vs Voltage")
    plt.savefig(output_path+"/ResponseTimevsTemp/"+name_of_LC+"_"+temp+".jpg", dpi=300,bbox_inches='tight')
    plt.show()


In [ ]:
titles=['Rise Time','Decay Time','Total Response Time']
if not os.path.exists(output_path+"/ResponseTimeMap") :
    os.mkdir (output_path+"/ResponseTimeMap")
for title in titles:
    plt.figure(1,figsize=(12, 2), dpi=100, linewidth=0.1)
    teste = np.resize((results[title]*1000).to_numpy(copy=True),(len(temps),len(voltages)))
    tas=pd.DataFrame(teste,temps,voltages)
    sns.heatmap(tas, annot=True,cmap='coolwarm')
    plt.xlabel("Applied Voltage (in Volts)")
    #plt.ylabel("Temperature (in Celcius)")
    plt.title(name_of_LC+" - "+title+" (in ms)")
    plt.savefig(output_path+"/ResponseTimeMap/"+name_of_LC+"_"+title+".jpg", dpi=300,bbox_inches='tight')
    plt.show()


# Dilelectric Analysis

In [ ]:
result_DE= pd.DataFrame(columns = ['LC Name', 'Temperature','e_infi','e_stat','Dielectric_Strength','Relaxation_freq','Relaxation_time'])

In [ ]:
path_DE="/home/avinash/Desktop/Projects/Liquid_Crystal_Analysis/data/Batch4/Dielectric-spectroscopy/Doped"
if not os.path.exists(path_DE + "/Analysis") :
    os.mkdir (path_DE + "/Analysis")
output_path_DE=path_DE+"/Analysis"

filename='+5(49.2)'

DE=pd.read_csv(path_DE+"/"+filename+".CSV",skiprows=4)
DE=DE[:-1]
DE.columns=['Frequency','Capacitance','Dielectric_loss','Resistance','Reactance','Impedance']
DE=DE.astype('float64')

#DE.Frequency=np.log10(DE.Frequency)

endfreq=pd.Series(DE.Frequency[DE.Capacitance<0]).iloc[0]#index to control
print(endfreq)
DE=DE[DE.Frequency<endfreq]

#newD=1/(DE.Frequency*DE.Capacitance*DE.Resistance)

real_e=pd.Series(DE.Capacitance*thickness/epsilon/area)
imag_e =real_e*DE.Dielectric_loss

startfreq=DE.Frequency[(imag_e.idxmin())+00]#offset to control
print(startfreq)
DE=DE[DE.Frequency>startfreq]

real_e=pd.Series(DE.Capacitance*thickness/epsilon/area)
imag_e =real_e*DE.Dielectric_loss

idx = np.argwhere(np.diff(np.sign(real_e - imag_e))).flatten()
idx=idx[0]
print(idx)

idx1=np.argmax(imag_e)
print(idx1)

plt.figure(3,figsize=(12, 8), dpi=100, linewidth=0.1)
plt.ylabel("Permitivity")
plt.xlabel("Frequency in Hz")
plt.plot(DE.Frequency,real_e)
plt.plot(DE.Frequency,imag_e)
plt.plot(DE.Frequency.iloc[idx1], imag_e.iloc[idx1], 'ro')
plt.annotate("Relaxation Frequency = "+str(round(DE.Frequency.iloc[idx1],2))+"Hz",(DE.Frequency.iloc[idx1], imag_e.iloc[idx1]),xytext =(DE.Frequency.iloc[idx1+5], imag_e.iloc[idx1]))
plt.title(name_of_LC+" : "+filename+" Permitivity vs Frequency")
plt.legend(['Real Component','Imaginary Component'])
if not os.path.exists(output_path_DE + "/Permitivity_VS_Frequency") :
    os.mkdir (output_path_DE + "/Permitivity_VS_Frequency")
plt.savefig(output_path_DE+"/Permitivity_VS_Frequency/"+filename+".jpg", dpi=300,bbox_inches='tight')

plt.figure(4,figsize=(12, 8), dpi=100, linewidth=0.1)
plt.xlabel("Real Permitivity")
plt.ylabel("Imaginary Permitivity")
plt.plot(real_e,imag_e)
plt.plot(real_e.iloc[idx1],imag_e.iloc[idx1], 'ro')
plt.annotate("Relaxation Frequency = "+str(round(DE.Frequency.iloc[idx1],2))+"Hz",(real_e.iloc[idx1],imag_e.iloc[idx1]),xytext =(real_e.iloc[idx1],imag_e.iloc[idx1]+0.1))
plt.title(name_of_LC+" : "+filename+" Cole-Cole Plot")
if not os.path.exists(output_path_DE + "/Cole-Cole_Plot") :
    os.mkdir (output_path_DE + "/Cole-Cole_Plot")
plt.savefig(output_path_DE+"/Cole-Cole_Plot/"+filename+".jpg", dpi=300,bbox_inches='tight')
#plt.show()
#print(1/(DE.Resistance*DE.Capacitance*DE.Frequency*pi*2),DE.Dielectric_loss)

# Analysing properties of the Liquid Crystal

In [ ]:
#Setting up dataframe for recording derived properties 
properties=pd.DataFrame(columns = ['Property', 'Value', 'Units'])

In [ ]:
#If the above Property is fine, append the property to list of properties
properties=properties.append({'Property':prop, 'Value':value, 'Units':units},ignore_index=True)
properties.tail()

In [ ]:
# If all the properties are fine, write out the properties to a csv file
properties.to_csv(output_path+"/"+"Derived_Properties.csv", index=False)

In [ ]:
#Calculating K from Threshold Voltage
#K depends on temperature
k_by_dielectric_anisotropy=(vth*2/pi)**2*epsilon
print("Combined elastic constant K divided by dielectric anisotropy "+"at temp "+str(temp)+" celcius = "+str(k_by_dielectric_anisotropy))

In [ ]:
#Calulating Rotational Viscocity by Dielectric Anisotrpy from Rise Time
gamma_by_DA=[]
for i in range(21,28):
    gamma_by_DA.append(results['Rise Time'][i]*epsilon*((results['Applied Voltage'][i])**2-vth**2)/thickness**2)
    print(results['Applied Voltage'][i],gamma_by_DA[i-21])

In [ ]:
#Comparing Visco Elasitc Ratios
for i in range(21,28):
    print(results['Visco-Elastic Ratio'][i],gamma_by_DA[i-21]/k_by_dielectric_anisotropy,results['Visco-Elastic Ratio'][i]-gamma_by_DA[i-21]/k_by_dielectric_anisotropy)